In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import math
from collections import Counter
from sklearn import mixture
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from skimage import measure

In [ ]:
from glob import glob
import os
from multiprocessing import Pool, cpu_count
from functools import partial
from subprocess import check_output

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.utils import np_utils
from keras import backend as K
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

# ensures image ordering adheres to tensorflow
K.set_image_dim_ordering('tf')

## Reading Data In

In [2]:
print(check_output(["ls", "../train"]).decode("utf-8"))

Type_1
Type_2
Type_3



In [3]:
TRAIN_DATA = "../train"

In [4]:
types = ['Type_1', 'Type_2', 'Type_3']

In [6]:
%%time
# creates list from image ids found in filenames in each type folder 
type_ids = []

for type in enumerate(types):
    type_i_files = glob(os.path.join(TRAIN_DATA, type[1], "*.jpg"))
    type_i_ids = np.array([s[len(TRAIN_DATA)+8:-4] for s in type_i_files])
    type_ids.append(type_i_ids)

['../train/Type_1/864.jpg']
['864']
['../train/Type_2/235.jpg']
['235']
['../train/Type_3/235.jpg']
['235']
CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 12.8 ms


In [7]:
def get_filename(image_id, image_type):
    """
    Method to grab image file path from its id and tpye
    """
    if image_type == "Type_1" or \
        image_type == "Type_2" or \
        image_type == "Type_3":
        data_path = os.path.join(TRAIN_DATA, image_type)
    else:
        raise Exception("Image type {} is not recognized".format(image_type))
        
    ext = 'jpg'
    return os.path.join(data_path, "{}.{}".format(image_id, ext))

In [8]:
def get_image_data(image_id, image_type):
    """
    Method to get image data as np.array that specifies image id and type
    """
    fname = get_filename(image_id, image_type)
    img = cv2.imread(fname)
    assert img is not None, "Failed to read image: {}, {}".format(image_id, image_type)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [9]:
# Crop images that may have circular frames present
# Find the largest inscribed rectangled in the thresholded image
# image is cropped to that rectangle

def maxHist(hist):
    maxArea = (0, 0, 0)
    height = []
    position = []
    for i in range(len(hist)):
        if (len(height) == 0):
            if (hist[i] > 0):
                height.append(hist[i])
                position.append(i)
        else: 
            if (hist[i] > height[-1]):
                height.append(hist[i])
                position.append(i)
            elif (hist[i] < height[-1]):
                while (height[-1] > hist[i]):
                    maxHeight = height.pop()
                    area = maxHeight * (i-position[-1])
                    if (area > maxArea[0]):
                        maxArea = (area, position[-1], i)
                    last_position = position.pop()
                    if (len(height) == 0):
                        break
                position.append(last_position)
                if (len(height) == 0):
                    height.append(hist[i])
                elif(height[-1] < hist[i]):
                    height.append(hist[i])
                else:
                    position.pop()    
    while (len(height) > 0):
        maxHeight = height.pop()
        last_position = position.pop()
        area =  maxHeight * (len(hist) - last_position)
        if (area > maxArea[0]):
            maxArea = (area, len(hist), last_position)
    return maxArea

In [10]:
def maxRect(img):
    maxArea = (0, 0, 0)
    addMat = np.zeros(img.shape)
    for r in range(img.shape[0]):
        if r == 0:
            addMat[r] = img[r]
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
        else:
            addMat[r] = img[r] + addMat[r-1]
            addMat[r][img[r] == 0] *= 0
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
    return (int(maxArea[3]+1-maxArea[0]/abs(maxArea[1]-maxArea[2])), maxArea[2], maxArea[3], maxArea[1], maxArea[0])

In [11]:
def cropCircle(img):
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    img = cv2.resize(img, dsize=tile_size)
            
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY);
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

    _, contours, _ = cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    main_contour = sorted(contours, key = cv2.contourArea, reverse = True)[0]
            
    ff = np.zeros((gray.shape[0],gray.shape[1]), 'uint8') 
    cv2.drawContours(ff, main_contour, -1, 1, 15)
    ff_mask = np.zeros((gray.shape[0]+2,gray.shape[1]+2), 'uint8')
    cv2.floodFill(ff, ff_mask, (int(gray.shape[1]/2), int(gray.shape[0]/2)), 1)
    
    rect = maxRect(ff)
    rectangle = [min(rect[0],rect[2]), max(rect[0],rect[2]), min(rect[1],rect[3]), max(rect[1],rect[3])]
    img_crop = img[rectangle[0]:rectangle[1], rectangle[2]:rectangle[3]]
    cv2.rectangle(ff,(min(rect[1],rect[3]),min(rect[0],rect[2])),(max(rect[1],rect[3]),max(rect[0],rect[2])),3,2)
    
    return [img_crop, rectangle, tile_size]

In [12]:
# For initial delineation of cervix, use two features
    # a color: the higher the a value, the redder the pixel color
    # R: distance of a pixel from image center -> spatial information
    #    -> support extraction of continuous regions

def Ra_space(img, Ra_ratio, a_threshold):
    imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB);
    w = img.shape[0]
    h = img.shape[1]
    Ra = np.zeros((w*h, 2))
    for i in range(w):
        for j in range(h):
            R = math.sqrt((w/2-i)*(w/2-i) + (h/2-j)*(h/2-j))
            Ra[i*h+j, 0] = R
            Ra[i*h+j, 1] = min(imgLab[i][j][1], a_threshold)
            
    Ra[:,0] /= max(Ra[:,0])
    Ra[:,0] *= Ra_ratio
    Ra[:,1] /= max(Ra[:,1])

    return Ra

In [13]:
# image is separated into two clusters in the 2-D (a-R) features space
# use of Gaussian mixture modeling
# initialized by k-means procedure

def get_and_crop_image(image_id, image_type):
    img = get_image_data(image_id, image_type)
    initial_shape = img.shape
    [img, rectangle_cropCircle, tile_size] = cropCircle(img)
    imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB);
    w = img.shape[0]
    h = img.shape[1]
    Ra = Ra_space(imgLab, 1.0, 150)
    a_channel = np.reshape(Ra[:,1], (w,h))
    
    g = mixture.GaussianMixture(n_components = 2, covariance_type = 'diag', random_state = 0, init_params = 'kmeans')
    image_array_sample = shuffle(Ra, random_state=0)[:1000]
    g.fit(image_array_sample)
    labels = g.predict(Ra)
    labels += 1 # Add 1 to avoid labeling as 0 since regionprops ignores the 0-label.
    
    # The cluster that has the highest a-mean is selected.
    labels_2D = np.reshape(labels, (w,h))
    gg_labels_regions = measure.regionprops(labels_2D, intensity_image = a_channel)
    gg_intensity = [prop.mean_intensity for prop in gg_labels_regions]
    cervix_cluster = gg_intensity.index(max(gg_intensity)) + 1

    mask = np.zeros((w * h,1),'uint8')
    mask[labels==cervix_cluster] = 255
    mask_2D = np.reshape(mask, (w,h))

    cc_labels = measure.label(mask_2D, background=0)
    regions = measure.regionprops(cc_labels)
    areas = [prop.area for prop in regions]

    regions_label = [prop.label for prop in regions]
    largestCC_label = regions_label[areas.index(max(areas))]
    mask_largestCC = np.zeros((w,h),'uint8')
    mask_largestCC[cc_labels==largestCC_label] = 255

    img_masked = img.copy()
    img_masked[mask_largestCC==0] = (0,0,0)
    img_masked_gray = cv2.cvtColor(img_masked, cv2.COLOR_RGB2GRAY);
            
    _,thresh_mask = cv2.threshold(img_masked_gray,0,255,0)
            
    kernel = np.ones((11,11), np.uint8)
    thresh_mask = cv2.dilate(thresh_mask, kernel, iterations = 1)
    thresh_mask = cv2.erode(thresh_mask, kernel, iterations = 2)
    _, contours_mask, _ = cv2.findContours(thresh_mask.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    main_contour = sorted(contours_mask, key = cv2.contourArea, reverse = True)[0]
    cv2.drawContours(img, main_contour, -1, 255, 3)
    
    x,y,w,h = cv2.boundingRect(main_contour)
    
    rectangle = [x+rectangle_cropCircle[2],
                 y+rectangle_cropCircle[0],
                 w,
                 h,
                 initial_shape[0],
                 initial_shape[1],
                 tile_size[0],
                 tile_size[1]]

    return [image_id, img, rectangle]

In [14]:
def parallelize_image_cropping(image_ids):
    out = open('rectangles.csv', "w")
    out.write("image_id,type,x,y,w,h,img_shp_0_init,img_shape1_init,img_shp_0,img_shp_1\n")
    imf_d = {}
    p = Pool(cpu_count())
    for type in enumerate(types):
        print(type)
        partial_get_and_crop = partial(get_and_crop_image, image_type = type[1])    
        ret = p.map(partial_get_and_crop, image_ids[type[0]])
        for i in range(len(ret)):
            out.write(image_ids[type[0]][i])
            out.write(',' + str(type[1]))
            out.write(',' + str(ret[i][2][0]))
            out.write(',' + str(ret[i][2][1]))
            out.write(',' + str(ret[i][2][2]))
            out.write(',' + str(ret[i][2][3]))
            out.write(',' + str(ret[i][2][4]))
            out.write(',' + str(ret[i][2][5]))
            out.write(',' + str(ret[i][2][6]))
            out.write(',' + str(ret[i][2][7]))
            out.write('\n')
            img = get_image_data(image_ids[type[0]][i], type[1])
            if(img.shape[0] > img.shape[1]):
                tile_size = (int(img.shape[1]*256/img.shape[0]), 256)
            else:
                tile_size = (256, int(img.shape[0]*256/img.shape[1]))
            img = cv2.resize(img, dsize=tile_size)
            cv2.rectangle(img,
                          (ret[i][2][0], ret[i][2][1]), 
                          (ret[i][2][0]+ret[i][2][2], ret[i][2][1]+ret[i][2][3]),
                          255,
                          2)
            #plt.imshow(img)
            #plt.show()
        ret = []
    out.close()

    return

In [15]:
%%time
parallelize_image_cropping(type_ids)

(0, 'Type_1')
(1, 'Type_2')
(2, 'Type_3')
CPU times: user 37min 21s, sys: 1min 39s, total: 39min 1s
Wall time: 25min 35s


Process ForkPoolWorker-3:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-1:
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/ubuntu/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/lib/py

## Resize Images

In [16]:
# import resizing dimensions from newly created csv
df = pd.read_csv('rectangles.csv')

In [17]:
df.head(5)

,image_id,type,x,y,w,h,img_shp_0_init,img_shape1_init,img_shp_0,img_shp_1
0,864,Type_1,1,1,190,253,4160,3120,192,256
1,2234,Type_1,1,1,190,253,4128,3096,192,256
2,2112,Type_1,1,39,176,176,3264,2448,192,256
3,1704,Type_1,1,32,166,178,4160,3120,192,256
4,1346,Type_1,17,58,156,151,4160,3120,192,256


In [18]:
# all files are accounted for
df.shape

(2615, 10)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2615 entries, 0 to 2614
Data columns (total 10 columns):
image_id           2615 non-null int64
type               2615 non-null object
x                  2615 non-null int64
y                  2615 non-null int64
w                  2615 non-null int64
h                  2615 non-null int64
img_shp_0_init     2615 non-null int64
img_shape1_init    2615 non-null int64
img_shp_0          2615 non-null int64
img_shp_1          2615 non-null int64
dtypes: int64(9), object(1)
memory usage: 204.4+ KB


In [20]:
# concatenate all image paths into a single list
all_files = []

for type in enumerate(types):
    all_files.append(glob(os.path.join(TRAIN_DATA, type[1], "*.jpg")))

all_files = np.concatenate(all_files)

In [21]:
# create a dictionary: key -> image path, value -> resizing dimensions

item_dict = {}
for item in all_files:
    item_idx = int(item.split('/')[-1][:-4])
    type_idx = item.split('/')[2]
    
    match = df[(df['image_id'] == item_idx) & (df['type'] == type_idx)]

    xd = int(match['x'].item())
    yd = int(match['y'].item())
    wd = int(match['w'].item())
    hd = int(match['h'].item())
    img_shp_0d = int(match['img_shp_0'].item())
    img_shp_1d = int(match['img_shp_1'].item())
    type_d = match['type'].item()
    
    item_dict[item_idx, type_d] = (xd, yd, wd, hd, img_shp_0d, img_shp_1d, type_d) 
    # xd:0, yd:1, wd:2, hd:3, img_0d:4, img_1d:5

In [157]:
%%time

labels = []
segmented_images = []
def generate_labels_and_images(img_dims):
    for item in all_files:

        item_ind = int(item.split('/')[-1][:-4]) # grab image id from filename
        type_ind = item.split('/')[2]            # grab type from filename -> use both values as a primary key
        img = cv2.imread(item)                   # use cv2 module to read in image data

        # return relevant resizing information from dict using primary key combo
        x_i, y_i, w_i, h_i, img_shp_0_i, img_shp_1_i, type_i = item_dict[item_ind, type_ind]

        labels.append(type_i)

        # resize to new image dimensions
        resized = cv2.resize(img, (img_shp_0_i, img_shp_1_i))

        # crop image to segmented area
        crop_img = resized[y_i:y_i+h_i, x_i:x_i+w_i]

        # resize image, passed in as argument
        desired_size = img_dims
        old_size = crop_img.shape[:2]
        
        # use old image size and new dimensions to scale image down

        ratio = float(desired_size)/max(old_size)
        new_size = tuple([int(x*ratio) for x in old_size])

        im = cv2.resize(crop_img, (new_size[1], new_size[0]))

        delta_w = desired_size - new_size[1]
        delta_h = desired_size - new_size[0]
        top, bottom = delta_h//2, delta_h-(delta_h//2)
        left, right = delta_w//2, delta_w-(delta_w//2)
        color = [0, 0, 0]
        
        # add a black border to images that are not perfectly square
        new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=color)

        segmented_images.append(new_im)
        
        return segmented_images, labels

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.08 ms


In [ ]:
segmented_images, lables = generate_labels_and_images(img_dims=64)

In [131]:
len(segmented), len(labels)

(2615, 2615)

In [133]:
Counter(labels)

Counter({'Type_1': 710, 'Type_2': 1000, 'Type_3': 905})

## Data Pre-Processing

In [137]:
# Convert string labels to numbers
le = preprocessing.LabelEncoder()
le.fit(labels)
print(le.classes_)
labels = le.transform(labels)
Counter(labels)

LabelEncoder()

In [142]:
# split data -> stratified by label -> images as X, labels as y
X_train, X_test, y_train, y_test = train_test_split(segmented_images, 
                                                    labels, 
                                                    test_size=0.20, 
                                                    random_state=42,
                                                    stratify=labels)

In [143]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [144]:
X_train = X_train.reshape(X_train.shape[0], 64, 64, 3).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 64, 64, 3).astype('float32')

In [145]:
# divide by 255 to ensure image pixel values are between 0 and 1
X_train = X_train / 255.0
X_test = X_test / 255.0

In [146]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train, num_classes=3)
y_test = np_utils.to_categorical(y_test, num_classes=3)
num_classes = y_test.shape[1]

In [147]:
X_test.shape, X_train.shape

((523, 64, 64, 3), (2092, 64, 64, 3))

In [148]:
y_test.shape, y_train.shape

((523, 3), (2092, 3))

In [ ]:
# built-in keras image preprocessing
datagen = ImageDataGenerator(rotation_range=30,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             zoom_range=0.2,
                             fill_mode='nearest')

In [149]:
datagen.fit(X_train)

In [228]:
class BuildModel():
    
    def __init__(self, filepath=None):
        self.filepath = filepath
        
        
    def set_checkpoint(self, filepath):
        self.checkpoint = ModelCheckpoint(self.filepath, 
                                          monitor='val_loss', 
                                          verbose=2, 
                                          save_best_only=True, 
                                          mode='min')
        return [self.checkpoint]
        
    def set_callbacks_list(self):
        self.callbacks_list = self.set_checkpoint()

    def get_model(self):
        pass
    
    def fit_model(self, callback_lists):
        self.get_model().fit_generator(datagen.flow(X_train, y_train, batch_size=1), 
                                       validation_data=(X_test, y_test), 
                                       steps_per_epoch=len(X_train), 
                                       epochs=10, 
                                       callbacks=callback_lists, 
                                       verbose=1)

## Build Basic CNN

In [229]:
class BaseCNN(BuildModel):
   
    def __init__(self, filepath):
        self.filepath = filepath
        
    def get_checkpoint(self):
        return super().set_checkpoint(self.filepath)

    def get_model(self):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3),
                         activation='relu',
                         input_shape=(64,64,3),
                         data_format="channels_last"))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(3, activation='softmax'))

        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model
    
    def fit_model(self):
        super().fit_model(self.get_checkpoint())

In [230]:
base = BaseCNN('basecnn-best-weight.hdf5').fit_model()

Epoch 1/10
2092/2092 [==============================] - 178s - loss: 1.1261 - acc: 0.3547 - val_loss: 1.0923 - val_acc: 0.3824
Epoch 2/10
  83/2092 [>.............................] - ETA: 165s - loss: 1.0997 - acc: 0.3976

KeyboardInterrupt: 

## Alex-Net

In [ ]:
class AlexNet(BuildModel):
   
    def __init__(self, filepath):
        self.filepath = filepath
        
    def get_checkpoint(self):
        return super().set_checkpoint(self.filepath)

    def get_model():
        model = Sequential()
        model.add(Conv2D(96, kernel_size=(3,3),
                         strides=(2,2),
                         activation='relu',
                         padding='same',
                         input_shape=(64,64,3),
                         data_format="channels_last"))
        model.add(MaxPooling2D(pool_size=(2,2),
                               strides=(2,2)))
        model.add(BatchNormalization())

        model.add(Conv2D(256, (5,5), activation='relu', padding='same'))
        model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
        model.add(BatchNormalization())

        model.add(Conv2D(384, (3,3), activation='relu', padding='same'))
        model.add(Conv2D(384, (3,3), activation='relu', padding='same'))
        model.add(Conv2D(256, (3,3), activation='relu', padding='same'))
        model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
        model.add(BatchNormalization())

        model.add(Flatten())
        model.add(Dense(4096, activation='tanh'))
        model.add(Dropout(0.5))
        model.add(Dense(4096, activation='tanh'))
        model.add(Dropout(0.5))
        model.add(Dense(3, activation='softmax'))

        # Compile model
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

        return model
    
    def fit_model(self):
        super().fit_model(self.get_checkpoint())

In [55]:
alex = AlexNet('alexnet-best-weight.hdf5').fit_model()

## VGG-16 Net

In [ ]:
class VGG16Net(BuildModel):
   
    def __init__(self, filepath):
        self.filepath = filepath
        
    def get_checkpoint(self):
        return super().set_checkpoint(self.filepath)

    def vgg16_net():
        model = Sequential()
        model.add(ZeroPadding2D((1,1),input_shape=(64, 64, 3), data_format='channels_last'))
        model.add(Conv2D(filters=64, kernel_size=3, strides=(3, 3), activation='relu', padding='same'))
        model.add(ZeroPadding2D(padding=(1,1), data_format='channels_last'))
        model.add(Conv2D(filters=64, kernel_size=3, strides=(3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D((2,2), strides=(2,2), padding='same'))

        model.add(ZeroPadding2D((1,1)))
        model.add(Conv2D(filters=128, kernel_size=3, strides=(3, 3), activation='relu', padding='same'))
        model.add(ZeroPadding2D((1,1)))
        model.add(Conv2D(filters=128, kernel_size=3, strides=(3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

        model.add(ZeroPadding2D((1,1)))
        model.add(Conv2D(filters=256, kernel_size=3, strides=(3, 3), activation='relu', padding='same'))
        model.add(ZeroPadding2D((1,1)))
        model.add(Conv2D(filters=256, kernel_size=3, strides=(3, 3), activation='relu', padding='same'))
        model.add(ZeroPadding2D((1,1)))
        model.add(Conv2D(filters=256, kernel_size=3, strides=(3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D((2,2), strides=(2,2), padding='same'))

        model.add(ZeroPadding2D((1,1)))
        model.add(Conv2D(filters=512, kernel_size=3, strides=(3, 3), activation='relu', padding='same'))
        model.add(ZeroPadding2D((1,1)))
        model.add(Conv2D(filters=512, kernel_size=3, strides=(3, 3), activation='relu', padding='same'))
        model.add(ZeroPadding2D((1,1)))
        model.add(Conv2D(filters=512, kernel_size=3, strides=(3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D((2,2), strides=(2,2), padding='same'))

        model.add(ZeroPadding2D((1,1)))
        model.add(Conv2D(filters=512, kernel_size=3, strides=(3, 3), activation='relu', padding='same'))
        model.add(ZeroPadding2D((1,1)))
        model.add(Conv2D(filters=512, kernel_size=3, strides=(3, 3), activation='relu', padding='same'))
        model.add(ZeroPadding2D((1,1)))
        model.add(Conv2D(filters=512, kernel_size=3, strides=(3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D((2,2), strides=(2,2), padding='same'))

        model.add(Flatten())
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(4096, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(3, activation='softmax'))

        sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(optimizer=sgd, loss='categorical_crossentropy')

        return model
    
    def fit_model(self):
        super().fit_model(self.get_checkpoint())

In [ ]:
vgg16 = VGG_16('vgg16net-best-weight.hdf5').fit_model()